In [1]:
import pandas as pd

## Data Import
* Here we use a simple spreadsheet format.
* The data has been processed to achieve the following format, which works for our functions.
* You can generate a similar format, otherwise you can adjust our function accordingly.

In [3]:
equity = pd.read_excel('FCFF_FCFE_format.xlsx', header = 1).fillna(0)

equity = equity.drop(equity.columns[0], axis = 1)

equity = equity.set_index('Years')

equity = equity.T

equity

Years,EBIT,Taxes,Depreciation & Amortization,CapEx,Increase in Working Capital,Increase in net other assets,Net Income,Preferred Dividends,Principal repayments,Proceeds from new debt issues
1,660.0000,112.2000,0.0,100.00000,45.0000,0.0,495.0000,0.0,0.0,72.5000
2,647.4600,110.0682,0.0,99.00000,44.5500,0.0,503.5800,0.0,0.0,71.7750
3,621.5616,105.6655,0.0,95.92000,43.1640,0.0,505.0188,0.0,0.0,69.5420
4,581.9370,98.9293,0.0,90.64440,40.7900,0.0,498.8032,0.0,0.0,65.7172
5,528.7314,89.8843,0.0,83.13386,37.4102,0.0,484.6704,0.0,0.0,60.2721


## Defining the function

In [5]:
def equity_fcff_fcfe(df: pd.DataFrame, g_fcff: float, g_fcfe: float, shares_outstanding: int, 
                debt_value: float, preferred_dividends: float,
                re: float, rd: float,
                debt_ratio: float, tax_rate: float):
    
    # Ensure correct column names are used
    required_columns = ['EBIT', 'Taxes', 'Depreciation & Amortization', 'CapEx', 
                        'Increase in Working Capital', 'Increase in net other assets',
                        'Net Income', 'Preferred Dividends', 'Principal repayments', 
                        'Proceeds from new debt issues']

    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found in DataFrame.")

    # Initialize results storage
    FCFF_values = []
    FCFE_values = []
    PVIF_FCFF_values = []
    PVIF_FCFE_values = []
    
    # Extract constant growth rates and WACC
    WACC = (re * (1 - debt_ratio)) + (rd * (1 - tax_rate) * debt_ratio)
    g_FCFF = g_fcff
    g_FCFE = g_fcfe
    
    # Loop through each row in the DataFrame and calculate FCFF and FCFE
    for i in range(len(df)):
        year = df.index[i]
        
        # Access values for the given year
        EBIT = df['EBIT'].iloc[i]
        taxes = df['Taxes'].iloc[i]
        depreciation_amortization = df['Depreciation & Amortization'].iloc[i]
        capex = df['CapEx'].iloc[i]
        working_capital = df['Increase in Working Capital'].iloc[i]
        net_other_assets = df['Increase in net other assets'].iloc[i]
        
        # NOPAT calculation
        NOPAT = EBIT - taxes
        
        # Gross Cash Flows
        gross_cash_flows_FCFF = NOPAT + depreciation_amortization
        
        # FCFF Calculation
        FCFF = gross_cash_flows_FCFF - working_capital - capex - net_other_assets
        FCFF_values.append(FCFF)
        
        # FCFE Calculation
        net_income = df['Net Income'].iloc[i]
        preferred_dividends = df['Preferred Dividends'].iloc[i]
        principal_repayments = df['Principal repayments'].iloc[i]
        new_debt_issues = df['Proceeds from new debt issues'].iloc[i]
        
        # Gross Cash Flows for FCFE
        gross_cash_flows_FCFE = net_income + depreciation_amortization
        
        # FCFE Calculation
        FCFE = (gross_cash_flows_FCFE - working_capital - capex - net_other_assets - 
                preferred_dividends - principal_repayments + new_debt_issues)
        FCFE_values.append(FCFE)
        
        # PVIF Calculation using 'Year' directly from the DataFrame
        PVIF_FCFF = 1 / ((1 + WACC) ** year)
        PVIF_FCFF_values.append(PVIF_FCFF)

        PVIF_FCFE = 1 / ((1 + re) ** year)
        PVIF_FCFE_values.append(PVIF_FCFE)
    
    # Calculate Terminal Values
    terminal_value_FCFF = FCFF_values[-1] * (1 + g_FCFF) / (WACC - g_FCFF)
    terminal_value_FCFE = FCFE_values[-1] * (1 + g_FCFE) / (re - g_FCFE)
    
    # Present Value of FCFF and FCFE
    pv_fcff = sum(FCFF * PVIF_FCFF for FCFF, PVIF_FCFF in zip(FCFF_values, PVIF_FCFF_values))
    pv_fcfe = sum(FCFE * PVIF_FCFE for FCFE, PVIF_FCFE in zip(FCFE_values, PVIF_FCFE_values))
    
    # Adding the present value of terminal values
    pv_terminal_value_FCFF = terminal_value_FCFF * PVIF_FCFF_values[-1]
    pv_terminal_value_FCFE = terminal_value_FCFE * PVIF_FCFE_values[-1]
    
    # Firm value and Equity value using FCFF method
    firm_value_fcff = pv_fcff + pv_terminal_value_FCFF
    equity_value_fcff = firm_value_fcff - debt_value
    
    # Equity value using FCFE method
    equity_value_fcfe = pv_fcfe + pv_terminal_value_FCFE
    
    # Stock Price calculations
    stock_price_fcff = equity_value_fcff / shares_outstanding
    stock_price_fcfe = equity_value_fcfe / shares_outstanding
    
    # Return the calculated values in the specified dictionary format, rounded to 6 decimals
    return {
        'FCFF Valuation': {
            'PV of FCFF': round(pv_fcff, 6),
            'PV of Terminal Value': round(pv_terminal_value_FCFF, 6),
            'PV of Firm': round(firm_value_fcff, 6),
            'PV of Debt': round(debt_value, 6),
            'PV of Equity': round(equity_value_fcff, 6),
            'Number of Shares Outstanding': round(shares_outstanding, 6),
            'Stock Price': round(stock_price_fcff, 6)
        },
        'FCFE Valuation': {
            'PV of FCFE': round(pv_fcfe, 6),
            'PV of Terminal Value': round(pv_terminal_value_FCFE, 6),
            'PV of Equity': round(equity_value_fcfe, 6),
            'Number of Shares Outstanding': round(shares_outstanding, 6),
            'Stock Price': round(stock_price_fcfe, 6)
        }
    }

## Applying the function on real data

In [8]:
equity_fcff_fcfe(equity,
                g_fcff = 0.03, g_fcfe = 0.015, shares_outstanding = 500, 
                debt_value = 1590, preferred_dividends = 0,
               re = 0.116, rd = 0.06,
               debt_ratio = 0.5, tax_rate = 0.17)

{'FCFF Valuation': {'PV of FCFF': 1473.949147,
  'PV of Terminal Value': 4161.793912,
  'PV of Firm': 5635.743059,
  'PV of Debt': 1590,
  'PV of Equity': 4045.743059,
  'Number of Shares Outstanding': 500,
  'Stock Price': 8.091486},
 'FCFE Valuation': {'PV of FCFE': 1562.960316,
  'PV of Terminal Value': 2463.754748,
  'PV of Equity': 4026.715064,
  'Number of Shares Outstanding': 500,
  'Stock Price': 8.05343}}

* Note that the prices using FCFF and FCFE can be slight different.
* It is essential to use real-world data to reconcile the two approaches, because if you want to test with arbitrary entries then it is not realistis due to the interplay components in the company structure, which are discussed in classical textbooks.